# Term Project - RQ1 Non-code Contributions

##### Install Dependencies

In [1]:
# install any missing dependencies (only needed if you haven't installed these already during tutorials)
!pip install "git+https://github.com/EPICLab/miner-utils"
!pip install 'gitpython'
!pip install 'pandas'
!pip install 'numpy'
!pip install 'matplotlib'
!pip install 'stop-words'

  Cloning https://github.com/EPICLab/miner-utils to c:\users\rupika\appdata\local\temp\pip-req-build-7wd1vu3a
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for MinerUtils: filename=MinerUtils-2.6.0-py3-none-any.whl size=7874 sha256=7354aa7697c1835b4c3b9a1fd15c5f8244315f7b52e3657a60fe6a08bf19c16b
  Stored in directory: C:\Users\Rupika\AppData\Local\Temp\pip-ephem-wheel-cache-qkpg5o5i\wheels\69\df\36\ed7f1fce1e25b44fbc7932df69094cd8b2f5d3d5c162ca4565
Successfully built MinerUtils


ERROR: Invalid requirement: "'gitpython'"
ERROR: Invalid requirement: "'pandas'"
ERROR: Invalid requirement: "'numpy'"
ERROR: Invalid requirement: "'matplotlib'"
ERROR: Invalid requirement: "'stop-words'"


##### GitHub Token Info

In [1]:
userName = 'rupikad'
token = '68e5fc89fbc9018314036e882450b759ca248244'

##### Imports

In [2]:
# setup environment (import any needed dependencies)
from minerutils import GitHub
import pandas as pd
import numpy as np
import datetime
import re
import matplotlib
import statistics
from scipy.stats import shapiro, wilcoxon
from sklearn import datasets, linear_model
from os import path
import json

# create github obj
gh = GitHub(userName, token)

##### JSON File I/O

In [3]:
# opens the json file if it exists
def open_file(file_name):
#     loc = '../proj_data/{}'.format(file_name)
#     loc = path.abspath(file_name)
#     print(loc)
#     file = path.exists(loc)
    file = path.exists(file_name)
#     print(file)
    json_data = None
    if file:
        with open(file_name, "r", encoding="utf8") as f:
           json_data = json.loads(f.read())
        return True, json_data
    else:
        return False, json_data

# create new json file if it doesn't exist
def create_file(data, file_name):   
    # write to json from api call data
    with open(file_name, "w+") as f:
        f.write(json.dumps(data))    
        gh.writeData(file_name, data)

    # read data from json file
    return gh.readData(file_name)

##### Saving 10 project data to JSON files
* Feeding in project urls
* Saving issue data for each url
* Saving issue comment data for each url
* Saving the file names for issue / issue comments in respective arrays to process the data later

In [4]:
# array of 10 projects
projs = ['d3/d3', 'EbookFoundation/free-programming-books', 'sindresorhus/awesome',
         'jwasham/coding-interview-university', 'getify/You-Dont-Know-JS', 'ohmyzsh/ohmyzsh', 
         'donnemartin/system-design-primer', 'github/gitignore','airbnb/javascript', 'public-apis/public-apis']

# array of issue / issue comments file names
issue_json, issue_cmt_json = ([] for i in range(2))

# getting all projects
for p in projs:
    # format strings
    url = '/repos/{}/issues'.format(p)
    cmt_url = '/repos/{}/issues/comments'.format(p)
    name = "{}_issues.json".format(p.split('/')[0])
    cmt_name = "{}_issue_cmt.json".format(p.split('/')[0])
    
    # read in issues and append name to array
    check, issues = open_file(name)
#     print('issue check; ', check)
    if not check:
        issues = gh.get(url, params={"since": "2015-01-01T:00:00:00Z", "state": 'all'})  
        issues = create_file(issues, name)
    issue_json.append(name)
     
    # read in issue comments and append name to array
    chk, comments = open_file(cmt_name)
#     print('cmt check; ', check)

    if not chk:
        comments = gh.get(cmt_url, params={"since": "2015-01-01T:00:00:00Z", "state": 'all'})  
        comments = create_file(comments, cmt_name)
    issue_cmt_json.append(cmt_name)
    
    print('read in issues + comments for project: {}\n'.format(p))

print(issue_json)
print(issue_cmt_json)

read in issues + comments for project: d3/d3

read in issues + comments for project: EbookFoundation/free-programming-books

read in issues + comments for project: sindresorhus/awesome

read in issues + comments for project: jwasham/coding-interview-university

read in issues + comments for project: getify/You-Dont-Know-JS

read in issues + comments for project: ohmyzsh/ohmyzsh

read in issues + comments for project: donnemartin/system-design-primer

read in issues + comments for project: github/gitignore

read in issues + comments for project: airbnb/javascript

read in issues + comments for project: public-apis/public-apis

['d3_issues.json', 'EbookFoundation_issues.json', 'sindresorhus_issues.json', 'jwasham_issues.json', 'getify_issues.json', 'ohmyzsh_issues.json', 'donnemartin_issues.json', 'github_issues.json', 'airbnb_issues.json', 'public-apis_issues.json']
['d3_issue_cmt.json', 'EbookFoundation_issue_cmt.json', 'sindresorhus_issue_cmt.json', 'jwasham_issue_cmt.json', 'getify_i

##### Saving Issue comment data for each project in a dictionary
* Iterating through projects from `issue_cmt_json` which is an array of projects
* Getting the counts for each issue comment author in a project
* Saving each project df into a dict `issue_comments`
* *NOTE: vue issue comments are not being fetched properly, excluded for now..*

In [5]:
# temp = ['airbnb_issue_cmt.json']

# dict for issue comments df
issue_comments = {}

# get the freq contributor list for issue comments
for c in issue_cmt_json:
# for c in temp:
    check, cmt = open_file(c)

    # create new df
    issue_comments_all = pd.DataFrame(cmt)

    # adding the commenter name to df
    issue_comments_all['login_name'] = issue_comments_all.user.apply(lambda x: x.get('login'))
    
    # store the login in a new df
    comment = issue_comments_all['login_name'].copy().to_frame('names')
    
    # add login to df
    issue_comments[c.split('_')[0]] = comment
  
len(issue_comments.keys())

10

#### Saving Issue data for each project in a dictionary
* Iterating through projects from `issue_json` which is an array of projects
* Getting the counts for each issue author in a project
* Saving each project df into a dict `issues_only`

In [6]:
# dict for issues df
issues_only = {}

# get freq for issue authors
for c in issue_json:
    check, issue = open_file(c)
    
    # create new df
    issues_all = pd.DataFrame(issue)
    
    # take all the rows where 'pull_request' is null to get only issues
    issues_all = issues_all[issues_all['pull_request'].isnull()]
    # reset the index
    issues_all.reset_index(drop=True, inplace=True)
    # drop PR column
    issues_all = issues_all.drop(columns=['pull_request'])

    # adding the issue founder name to df
    issues_all['login_name'] = issues_all.user.apply(lambda x: x.get('login'))
    
    # store the login in a new df
    author = issues_all['login_name'].copy().to_frame('names')

    # add login df to dict
    issues_only[c.split('_')[0]] = author

    
issues_only['d3']


,names
0,altocumulus
1,vijayakumarmariyappan
2,Badri-Tib
3,arfath261
4,goyaltushar92
...,...
1008,janwillemtulp
1009,janwillemtulp
1010,mbostock
1011,mbostock


#### Saving Merged_By + Timestamp
Amreeta -- 1) use second instead of first 2) check your path!

In [12]:
# first and second half of issue list to run...
# first = ['sindresorhus_issues.json', 'jwasham_issues.json', 'getify_issues.json', 'ohmyzsh_issues.json', 'donnemartin_issues.json', 'github_issues.json', 
#           'public-apis_issues.json', 'd3_issues']
# second = ['ohmyzsh_issues.json', 'donnemartin_issues.json', 'github_issues.json', 
#           'public-apis_issues.json']
third = ['d3_issues.json', 'EbookFoundation_issues.json']


# process merged_by for issue files
for m in third:
    print(m)
    check, issue_pr = open_file(m)
#     print(issue_pr[0])
    
    # display the ISSUES
    df_issue = pd.DataFrame(issue_pr)
    
#     print(df_issue.head(5))

    # drop all issues, keep PR and reset index
    all_pulls = df_issue.dropna(subset=['pull_request'])
    all_pulls.reset_index(drop=True, inplace=True)
    
    # intialize empty merge dict 
    # will look like: {index: [name, timestamp]}
    merged_by = {}

    # get all people who merged PR
    for i, row in all_pulls.iterrows():
        # get url, and then get the api call only
        # will look like '/repos/user/project/pulls/pull_number'
        url = all_pulls.loc[i, 'pull_request']['url'].split('com', 1)[1] 
        pr = gh.get(url)

        # append row to merge dict  
        if(pr['merged_by']):
            merged_by[i] = [pr['merged_by']['login'], pr['merged_at']]

        if i % 150 == 0:
            print('now on project {}, row:{}'.format(m, i))
            
    # convert dict into dataframe and reset index
    merge_data = pd.DataFrame.from_dict(merged_by, orient='index', columns=['name', 'merged_at'])
    merge_data.reset_index(drop=True, inplace=True)
    print(merge_data.head(10))

    # download df data into a json file
    loc = '../Term Project/merge_json/{}_merged.json'.format(m.split('_')[0])
    merge_data.to_json(loc, orient='records')

d3_issues.json
now on project d3_issues.json, row:0
now on project d3_issues.json, row:150
       name             merged_at
0       Fil  2020-08-26T20:38:10Z
1       Fil  2020-08-26T21:34:57Z
2  mbostock  2019-09-25T05:15:43Z
3  mbostock  2018-08-29T20:05:38Z
4  mbostock  2019-01-19T16:22:11Z
5  mbostock  2018-05-03T13:18:18Z
6  mbostock  2018-01-18T19:48:05Z
7  mbostock  2017-10-02T17:09:00Z
8  mbostock  2017-03-22T15:38:04Z
9  mbostock  2017-02-15T22:32:51Z
EbookFoundation_issues.json
now on project EbookFoundation_issues.json, row:0
now on project EbookFoundation_issues.json, row:150
now on project EbookFoundation_issues.json, row:300
now on project EbookFoundation_issues.json, row:450
now on project EbookFoundation_issues.json, row:600
now on project EbookFoundation_issues.json, row:750
now on project EbookFoundation_issues.json, row:900
now on project EbookFoundation_issues.json, row:1050
now on project EbookFoundation_issues.json, row:1200
now on project EbookFoundation_issues.j

### Run after Merge Files

We can run this once we have all the merge files ready..

In [7]:
# read from merge file and convert to df here and get freq here..
merge_file = ['d3_merged.json', 'EbookFoundation_merged.json', 'sindresorhus_merged.json', 'jwasham_merged.json', 
              'getify_merged.json', 'ohmyzsh_merged.json', 'donnemartin_merged.json', 'github_merged.json', 
              'airbnb_merged.json', 'public-apis_merged.json']

# dict for merged df
merged_names = {}

# read merged json data from file and save to dict
for f in merge_file:
    # get path and read in json
    loc = '../Term Project/merge_json/{}'.format(f)
    m_df = pd.read_json(loc, orient='records')
    
#     print(m_df)
    
    csv = '../Term Project/merge_csv/{}_timestamp.csv'.format(f.split('_')[0])
    m_df.to_csv(csv, index = False, header=True)
    
    # drop timestamp col and rename
    m_name = m_df.drop(columns='merged_at')
    m_name.columns = ['names']

    # add to dict
    merged_names[f.split('_')[0]] = m_name

merged_names.keys()

dict_keys(['d3', 'EbookFoundation', 'sindresorhus', 'jwasham', 'getify', 'ohmyzsh', 'donnemartin', 'github', 'airbnb', 'public-apis'])

In [8]:
merged_names['d3']

,names
0,Fil
1,Fil
2,mbostock
3,mbostock
4,mbostock
5,mbostock
6,mbostock
7,mbostock
8,mbostock
9,mbostock


In [9]:
# concat all the dfs!!!
proj_names = ['d3', 'EbookFoundation', 'sindresorhus', 'jwasham', 'getify', 'ohmyzsh', 
              'donnemartin', 'github', 'airbnb', 'public-apis']
# new dict
non_code = {}

# merge all the non-code data and get the ranking
for key in proj_names:
    all_data = pd.concat([merged_names[key], issues_only[key], issue_comments[key]], ignore_index=True)
    
    non_code[key] = all_data['names'].value_counts().to_frame().reset_index()
    non_code[key].columns = ['login', 'contributions']

non_code['d3']

,login,contributions
0,mbostock,1425
1,curran,76
2,jasondavies,67
3,Fil,47
4,mgold,31
...,...,...
1242,r1m,1
1243,qian99,1
1244,mantri-govind,1
1245,dfranco,1


In [10]:
# get the code contributors

# dict for storing code df
code = {}

for p in projs:
    url = '/repos/{}/contributors'.format(p)
    contributors = gh.get(url)
    
    key = p.split('/')[0]
    
    code[key] = pd.DataFrame(contributors)
    code[key] = code[key][['login', 'contributions']]
    

code

{'d3':             login  contributions
 0        mbostock           3085
 1     jasondavies            776
 2      kitmonisit             50
 3             Fil             44
 4        27359794             27
 ..            ...            ...
 116      ppeterka              1
 117        shilad              1
 118       vthiagu              1
 119   william-pan              1
 120  zanarmstrong              1
 
 [121 rows x 2 columns],
 'EbookFoundation':                 login  contributions
 0                 vhf           1541
 1             MHM5000            301
 2    alexanderfefelov            134
 3               MrS4w            130
 4             esparta            106
 ..                ...            ...
 406   sridhareaswaran              2
 407         ggcrunchy              2
 408               sru              2
 409   suzannehamilton              2
 410           sbagdat              2
 
 [411 rows x 2 columns],
 'sindresorhus':                login  contributions
 0  

In [11]:
# adding total % column for code dict

# create a new % dict?
code_rank = {}

for key, val in code.items():
    arr = val['contributions'].to_numpy()
    percent = arr/np.sum(arr)
    temp = np.round(percent*100, 3)

    t = pd.DataFrame(data = temp)
    t.columns = ['percentage']

    code_rank[key] = pd.concat([val, t], axis=1).drop(columns='contributions')
#     code_rank[key] = val

code_rank['d3']

,login,percentage
0,mbostock,72.537
1,jasondavies,18.246
2,kitmonisit,1.176
3,Fil,1.035
4,27359794,0.635
...,...,...
116,ppeterka,0.024
117,shilad,0.024
118,vthiagu,0.024
119,william-pan,0.024


In [12]:
# adding total % for non-code dict

# create a new % dict?
non_code_rank = {}

for key, val in non_code.items():
    arr = val['contributions'].to_numpy()
    percent = arr/np.sum(arr)
    temp = np.round(percent*100, 3)

    t = pd.DataFrame(data = temp)
    t.columns = ['percentage']

    non_code_rank[key] = pd.concat([val, t], axis=1).drop(columns='contributions')
#     non_code_rank[key] = val

non_code_rank['d3']

,login,percentage
0,mbostock,35.732
1,curran,1.906
2,jasondavies,1.680
3,Fil,1.179
4,mgold,0.777
...,...,...
1242,r1m,0.025
1243,qian99,0.025
1244,mantri-govind,0.025
1245,dfranco,0.025


In [18]:
# # create a new ranking list for all contributions
# nc = non_code_rank['d3'].copy()
# c = code_rank['d3'].copy()

# final dict
final_rank = {}

for key, val in code_rank.items():

    both = pd.merge(non_code_rank[key], val, how='outer', on='login').fillna(0).rename(columns = 
                                                                     {'percentage_x':'noncode%', 'percentage_y':'code%'})
    # unsure if this scoring is correct...
    both['total%'] = both['noncode%'] + both['code%']

    # get rankings for total% and code%
    both = both.sort_values('total%', ascending=False).reset_index(drop=True)
    both = both.sort_values('code%', ascending=False).reset_index().rename(columns={'index': 'Total Ranking'})
#     both = both.reset_index().rename(columns={'index': 'code% rank'}).drop(columns=['noncode%', 'code%', 'total%', 'login'])
    both = both.reset_index().rename(columns={'index': 'Github Ranking'})
    both[['Github Ranking', 'Total Ranking']] += 1
    
    final_rank[key] = both
    
    # write to csv
    csvv = '../Term Project/rq1_rank/{}_rank.csv'.format(key)
    both.to_csv(csvv, index = False, header=True)
    
    

# both.head(25)
len(final_rank['EbookFoundation'])


1088

In [19]:
# running wilcox test on Github + total ranking

wilcox_ranking = []

# running wilcox test on dict
for key, i in final_rank.items():
    git = i['Github Ranking'].to_numpy()
    total = i['Total Ranking'].to_numpy()

    s, p = wilcoxon(git, total)
    wilcox_ranking.append([key, s, round(p, 3), len(i)])
    
# convert list into df
final_wilcox = pd.DataFrame(wilcox_ranking, columns=['project', 'statistic', 'p-value', '# contributors'])
final_wilcox

,project,statistic,p-value,# contributors
0,d3,432120.0,0.429,1334
1,EbookFoundation,289623.0,0.560,1088
2,sindresorhus,395224.0,0.916,1262
3,jwasham,77099.0,0.990,561
4,getify,226160.0,0.459,966
5,ohmyzsh,10319569.5,0.851,6437
6,donnemartin,36522.0,0.910,387
7,github,409030.0,0.652,1295
8,airbnb,1132065.0,0.865,2139
9,public-apis,104735.0,0.005,694


In [8]:
# getting timestamps

# issue_comment_ts = pd.DataFrame()
# issue_ts = pd.DataFrame()


# get all issue comment timestamps
for c in issue_cmt_json:
    if not 'airbnb' in c and not 'getify' in c and not 'd3' in c:
        check, issue = open_file(c)

        # create new df
        issues_all = pd.DataFrame(issue)

        # store the ts in a new df and concat to timestamp
        ts = issues_all['created_at'].copy().to_frame('timestamp')
        
        csv = '../Term Project/timestamp/{}_issue_comment_ts.csv'.format(c.split('_')[0])
        ts.to_csv(csv, index = False, header=True)

#         issue_comment_ts = pd.concat([issue_comment_ts, ts], ignore_index=True)


# get all issue timestamps
for c in issue_json:
#     if 'airbnb' in c or 'getify' in c or 'd3' in c:
    if not 'airbnb' in c and not 'getify' in c and not 'd3' in c:
        check, issue = open_file(c)

        # create new df
        issues_all = pd.DataFrame(issue)

        # store the ts in a new df and concat to timestamp
        ts = issues_all['created_at'].copy().to_frame('timestamp')
        
        csv = '../Term Project/timestamp/{}_issues_ts.csv'.format(c.split('_')[0])
        ts.to_csv(csv, index = False, header=True)

#         issue_ts = pd.concat([issue_ts, ts], ignore_index=True)

# # save to csv
# issue_comment_ts.to_csv('../Term Project/timestamp/issue_comment_ts.csv', index = False, header=True)
# issue_ts.to_csv('../Term Project/timestamp/issue_ts.csv', index = False, header=True)   